In [2]:
import pandas as pd
import os 
import numpy as np

In [3]:
# model-specifics
model_specifics_dict = {}

model_specifics_dict["model_location_string"] = ["/home/daan/Documents/FATM_ABM/GAMA Model/models/main.gaml"]# the location of the GAMA model
model_specifics_dict["GAMA_location_string"] = ["/home/daan/GAMA"] # the location of GAMA main folder
model_specifics_dict["experiment_name"] = ["simple_simulation"] # experiment name as defined in the .gaml file
model_specifics_dict["stopping_condition"] = ["cycle=4320"] # stopping condition (> or < do not work)

model_specifics_df = pd.DataFrame(model_specifics_dict)
model_specifics_df.to_csv("model_specifics.csv")

## Construct experiment list
1. Permutation policy parametres
2. Permutation scenario parametres
3. Join both permutations
4. Export to .csv files with: 
    * name "input_parametres.csv"
    * parametre names as column names 
    

In [4]:
# state the policy options
capacity_policy         = [0,1,2] #[0]
minfood_access_policy   = [0]
ration_size_policy      = [30]#[15]
day_access_policy       = [0]
rerouting_policy        = [0,1,2,3] #[0,1,2,3]

refill_frequency  = [0]

# permutate the policy options
res = [[i, j, k, l , m,n]  for i in capacity_policy
                         for j in minfood_access_policy
                         for k in ration_size_policy
                         for l in day_access_policy
                         for m in rerouting_policy
                         for n in refill_frequency]

# List plausible uncertainty values (derived from qualitative scenario generation)
alpha_vals = [0.0,1.0]#[0.0]
beta_vals = [0.0,0.5,1.0]#,1.0] #[1.0]
gamma_vals = [3,7]
epsilon_vals = [0.1,0.5,1.0] 
psf_vals = [6.0,7.4] #[6.0,7.8]#[6.0,6.4,6.8,7.2,7.6,8.0,8.4,8.8]#,9.2,9.6,10]#,10.4,10.8,11.2,11.4,11.8]
int_vals = [20]

# Generate permutation
uncertainty_vals1 = np.array( [[i, j, k, l,m,n ] for i in alpha_vals
                                     for j in beta_vals
                                     for k in gamma_vals
                                     for l in epsilon_vals
                                     for m in psf_vals
                                     for n in int_vals])

reps = 1
parameter_values = list()
for i in range(reps): 
#     parameter_values.append([ 0. ,  0. , 15. ,  0. ,  0. ,  1. ,  0. ,  3. ,  0.1,  6. , 20. ])

    for u in uncertainty_vals1:
        for p in res:
            parameter_values.append(np.concatenate((p,u)))


        
# make a list of names of parameters representing policy options
policy_names = ["capacity_policy",
                "minfood_access_policy",
                "ration_size_policy",
                "day_access_policy",
                "rerouting_policy",
                "facility_refill_frequency"]
# Add scenario factor names to list of parameters
uncertainty_names = ["alpha","beta","gamma","epsilon","parallel_served_full","avg_interactions"]
parameter_names = policy_names+uncertainty_names
pd.DataFrame(parameter_values,columns=parameter_names).to_csv("input_parameters.csv")

In [5]:
len(parameter_values)

864

## Construct output list
1. Specify the outputs as monitors in the GAMA model
2. Specify the same outputs in one array
3. Export as csv file "output_names.csv"

In [6]:
output_names = list()
# List the names of the desired output
output_names.append("unsatisfied consumption")
output_names.append("food degraded")
output_names.append("distance covered")

output_names.append("food storage")
output_names.append("emotional state")

# 	monitor "food storage" value: facilities sum_of each.facility_food_storage;
# 	monitor "emotional state" value: households mean_of each.emotional_state;

q_string = "Queue length %s"
fs_string = "Food served %s" 

for i in range(12):
    output_names.append(q_string%i)
for j in range(12):
    output_names.append(fs_string%j)
pd.DataFrame(output_names).to_csv("output_names.csv")

## Run GAMA model in parallel using MPI  
1. mpiexec as MPI caller
2. -np m with m as number of threads
3. python specifying the python communicator
4. filename.py the independently runnable python script
5. os.system as bash communicator 

In [13]:
os.system("mpiexec -np 8 python MPI.py")

256

## Import results from Feather file
1. Exported to binary Feather in subdir "results"
2. Feather file named after experiment_name

In [89]:
results = pd.read_feather("results/%s"%model_specifics_dict["experiment_name"][0])

In [91]:
results.head(10)

,o0,o1,o2,o3,o4,o5,o6,o7,o8,o9,o10,o11,o12,o13,o14
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
